In [30]:
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import math as math

In [3]:
pip install --upgrade scipy==1.4.1

Note: you may need to restart the kernel to use updated packages.


In [231]:
def powerlaw(r, s0, b):  
    # r0 no es variable aleatoria
    r0 = 300
    return s0*(r/r0)**(-b)



def funcion_costo2(theta, xdata, ydata, ysigma):
        s0 = theta[0]
        b = theta[1]
        costo = np.zeros_like(s0)
        for (dist, y1, sigma1) in zip(xdata, ydata, ysigma):
            yfit = powerlaw(dist,s0,b)
            residuo = y1 - yfit
            z = (residuo / sigma1)**2
            costo += z
        return costo
    
costo2 = lambda theta: funcion_costo2(theta,r,y_random,sigma) 

def in_hex(poli):
    path = mpltPath.Path(poli)
    hex = False
    random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    while hex != True:
        if path.contains_points([random_point]) == True:
            hex = True
        else:
            random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    return random_point

def getDistances(xpositions, ypositions, x0=0, y0=0, zenithRad=0, azimuthRad=0):

    dx = xpositions - x0    # detector wrt core positions
    dy = ypositions - y0
    
    groundDistances = np.sqrt(dx*dx+dy*dy)

    # Direction cosines in the plane xy
    dirx =  math.cos(azimuthRad)
    diry =  math.sin(azimuthRad)

    axisProjections = math.sin(zenithRad) * (dx*dirx+dy*diry)
    
    distances = np.sqrt( groundDistances**2 - axisProjections**2 )
    
    return (distances,axisProjections,groundDistances)

def simZenith(zenithMinDeg, zenithMaxDeg) :

    zenithMinRad = math.radians(zenithMinDeg)
    sin2MinZenith = math.sin(zenithMinRad)**2

    zenithMaxRad = math.radians(zenithMaxDeg)
    sin2MaxZenith = math.sin(zenithMaxRad)**2

    rng = np.random.default_rng()
    u = rng.random() 
    
    sin2Zenith = sin2MinZenith + u * (sin2MaxZenith-sin2MinZenith)
    sinZenith = math.sqrt(sin2Zenith) 

    zenith = math.asin(sinZenith)
    azimut = np.random.uniform(0,2*np.pi)
    
    return (zenith,azimut)

poligono = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0]]
rectangulo = [[-433,-375],[433,-433],[433,375],[-433,375]]
pointsx,pointsy = [-216.5,216.5,433,216.5,-216.5,-433.5,0] , [-375,-375,0,375,375,0,0]
random_point = in_hex(poligono)
centro = (0,0)

In [7]:
print("Parametros ajustados: ", res.x)
cova = res.hess_inv
print("Covarianza estimada: ", 2*cova)  
par_error = np.sqrt(np.diagonal(2*cova))
print("Errores de los parámetros", par_error)

Parametros ajustados:  [1.45227346 0.79909061]
Covarianza estimada:  [[ 0.51954543 -0.20454545]
 [-0.20454545  0.08181818]]
Errores de los parámetros [0.720795   0.28603877]


In [232]:
#Defino parametros de LDF 
r0 = 300 #metros 
s0 = 35 #VEM
angulo = 40
b = 2.1    
#Genero el punto random
random_point = in_hex(poligono)
#calculo las distancias entre el evento y los 7 detectores.
distancias = getDistances(np.asarray(pointsx),np.asarray(pointsy),random_point[0],random_point[1],simZenith(-angulo,angulo)[0],simZenith(-angulo,angulo)[1])
r = np.asarray(distancias[0])


signal = powerlaw(r,s0,b)
sigma = np.sqrt(signal)

#Hago el ajuste y defino la semilla inicial
y_random = np.random.normal(signal,sigma,size = len(signal)) #signal
params=[35, 1]
popt, pcov = curve_fit(powerlaw, r, y_random, p0=params,sigma=sigma,absolute_sigma=True)
print('Curve_fit results')
print("Parametros ajustados: ",popt)
print("Errores de los parámetros", np.sqrt(np.diag(pcov)))
print(pcov)

Curve_fit results
Parametros ajustados:  [34.00306355  2.06489262]
Errores de los parámetros [2.55003054 0.13685325]
[[ 6.50265576 -0.21791854]
 [-0.21791854  0.01872881]]


In [233]:
res = minimize(costo2, x0=(30,1))
print('Minimize results')
print("Parametros ajustados: ", res.x)
cova = 2*res.hess_inv
par_error = np.sqrt(np.diagonal(cova))
print("Errores de los parámetros", par_error)
print(cova)

Minimize results
Parametros ajustados:  [34.00306549  2.06489246]
Errores de los parámetros [2.53449946 0.13294532]
[[ 6.4236875  -0.20676433]
 [-0.20676433  0.01767446]]
